In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [3]:
SHOW_SET = set(('New Zealand', 'China', 'Italy', 'Spain', 'Germany', 'US', 'Korea, South', 'Japan', 'Canada', 'Australia', 'Israel', 'Ireland', 'France', 'United Kingdom'))
SERIES = ['confirmed', 'deaths', 'recovered']
URL_FORMAT = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_%s_global.csv'
DOCS_LINK = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQCN9pL21lGy3XPBhKwMX7jT1_SG-Sb_4ZWZ1I0Ctd-0vNhtmH4gFKaLsV5jhz4vSjYlQ9NR_fXF_b6/pub?output=xlsx'

In [4]:
def update_nz_data(cdf, nz_df):
    nz_cdf = cdf.loc[(['confirmed', 'recovered', 'deaths'], 'New Zealand'), :nz_df.index[0]]
    #nz_cdf.iloc[:,:-1] = nz_cdf.iloc[:,1:].values
    cdf.loc[(['confirmed', 'recovered', 'deaths'], 'New Zealand'), nz_df.index[:-1]] = nz_df.loc[nz_df.index[1]:, ['Cases', 'Deaths', 'Recovered']].fillna(0).values.T

In [5]:
cdf =pd.concat({series: pd.read_csv(URL_FORMAT%series).groupby('Country/Region').sum().iloc[:,2:] for series in SERIES}, names=['Series'])
the_dates = pd.to_datetime(cdf.columns, dayfirst=False)
cdf.set_axis(the_dates, axis=1, inplace=True)
cdf.columns.rename('date', inplace=True)

In [6]:
with pd.ExcelFile(DOCS_LINK) as excel_file:
        nz_df = excel_file.parse('From Briefings', index_col='Date', parse_dates=True, header=1)
nz_df = nz_df[~nz_df.isna().all(1)]

In [7]:
update_nz_data(cdf, nz_df)

In [8]:
c30_idx = ['New Zealand'] + list(cdf.loc['confirmed', cdf.columns[-1]].nlargest(30).index.values)

In [9]:
cdf.loc['confirmed'].loc[c30_idx].pct_change(1, axis=1).iloc[:,-10:]

date,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04
Country/Region,,,,,,,,,,
New Zealand,0.300353,0.225543,0.139690,0.145914,0.098472,0.094281,0.125706,0.089084,0.094470,0.093684
US,0.274529,0.212570,0.194979,0.159766,0.148496,0.162941,0.133920,0.140979,0.131989,0.120703
Spain,0.167040,0.137282,0.114366,0.093876,0.097940,0.090579,0.085433,0.076327,0.063659,0.058465
Italy,0.083389,0.073323,0.069065,0.056417,0.041458,0.039837,0.045202,0.042216,0.039786,0.040099
Germany,0.177237,0.157791,0.134143,0.076263,0.077140,0.073604,0.084447,0.088889,0.075064,0.054114
France,0.154336,0.130317,0.140800,0.068311,0.109610,0.169515,0.093172,0.037750,0.087987,0.393331
China,0.001482,0.001406,0.001245,0.001500,0.000925,0.000985,0.000997,0.000862,0.000958,0.000388
Iran,0.088426,0.099504,0.095138,0.081931,0.083166,0.074949,0.066988,0.060408,0.053796,0.048136
United Kingdom,0.225311,0.248307,0.174093,0.142560,0.135137,0.134859,0.172050,0.144249,0.132151,0.097909


In [10]:
flat_df = cdf.stack().unstack('Series').loc[c30_idx]
flat_df = flat_df.assign(active=flat_df.confirmed-flat_df.deaths-flat_df.recovered)

In [11]:
def gen_rates(df, days_ago,names):
    today = df.iloc[:,-1]
    return pd.DataFrame({name:today/pair[1] for name,pair in zip(names, df.iloc[:, -1 - np.array(days_ago)].items())})

In [12]:
numdays_series = (cdf.loc['confirmed']>0).sum(1)
has10days = numdays_series.index[numdays_series>=10]


In [13]:
rates_all = gen_rates(cdf.loc['confirmed'],[1,3,7,14],['1 day', '3 day', '1 week', '2 week'])
rates = rates_all.loc[has10days]

In [14]:
def set_widths(figure, factor_series):
    for trace in figure.data:
        country = trace['name']
        factor = factor_series.loc[country]
        trace['hovertemplate'] = trace['hovertemplate'] + (''.join('<br>Scaling - %s=%.1fx' % kv for kv in factor.items()))
        if country not in SHOW_SET:
            trace['visible'] = 'legendonly'
        trace['line'].update(width=np.log2(factor['1 week'])+1)
    return figure

FIGURE_KW = dict(facet_col_wrap=2, height=900, labels={'0':'count'}, log_y=True, color='Country/Region', line_dash='Country/Region')
    
def coronavirus_figure(xdf, countries, today, factors, y=0, **kw):
    title='Coronavirus case data for NZ & the 30 countries with the most confirmed cases for %s' % today
    return set_widths(px.line(xdf, category_orders={'Country/Region':list(countries)}, x='date', y=y, title=title, **dict(FIGURE_KW, **kw)), factors)
    

In [15]:
rates.loc[c30_idx].nlargest(30, '1 week')

,1 day,3 day,1 week,2 week
Country/Region,,,,
Russia,1.140275,1.703637,3.742880,15.460784
Turkey,1.144018,1.526500,3.233450,35.722388
Brazil,1.143993,1.515506,2.653689,10.146915
US,1.120703,1.447472,2.542436,12.116992
Romania,1.135093,1.468699,2.488292,9.844687
United Kingdom,1.097909,1.422300,2.453616,8.383067
France,1.393331,1.573153,2.384149,6.281408
Canada,1.043499,1.357531,2.327475,10.162882
Poland,1.072125,1.420125,2.214286,6.766791


In [16]:
px.bar(rates.loc[c30_idx].unstack().reset_index(), barmode='group', color='Country/Region', x='level_0', y=0, log_y=True)

In [17]:
xdf = flat_df.stack().reset_index()
fig= coronavirus_figure(xdf, c30_idx, the_dates[-1].strftime('%d %B'), rates, facet_col='Series')
fig

In [18]:
xdf = flat_df.reset_index()
fig=coronavirus_figure(xdf, c30_idx, the_dates[-1].strftime('%e %B'), rates, y='confirmed', hover_data=['deaths', 'recovered', 'active'])
fig.write_html('../Coronavirus_plot.html', include_plotlyjs='cdn')
fig